In [ ]:
%load_ext autoreload
%autoreload 2

%config IPCompleter.greedy=True

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from astropy import units
from astropy.constants import G
import importlib

import dataset_compute
import snapshot_obj

In [ ]:
importlib.reload(dataset_compute)
importlib.reload(snapshot_obj)

In [ ]:
sim_id = "V1_LR_fix"
snap = snapshot_obj.Snapshot(sim_id, 127)
cops = snap.get_subhalos("CentreOfPotential") * units.cm.to(units.Mpc)
%time data = dataset_compute.group_particles_by_subhalo(snap, 'Coordinates', 'Masses', part_type=[1])
grouped_coords = data['Coordinates']

In [ ]:
print(data['GroupNumber'][56])
print(data['SubGroupNumber'][56])

In [ ]:
fig, ax = plt.subplots()
n = 5
for idx in range(n):
    coords = grouped_coords[idx] * units.cm.to(units.Mpc)
    ax.scatter(coords[:,0], coords[:,2])
ax.scatter(cops[:n,0], cops[:n,2], c='k')
ax.scatter(cops[55,0], cops[55,2], c='green')
LG_centre = dataset_compute.compute_LG_centre(snap) * units.cm.to(units.Mpc)
ax.scatter(LG_centre[0], LG_centre[2])
plt.show()

In [ ]:
gns = snap.get_subhalos("GroupNumber")
sgns = snap.get_subhalos("SubGroupNumber")
print(gns[514], sgns[514])

In [ ]:
%time cmass, radii = dataset_compute.mass_accumulation_to_array(snap)

In [ ]:
idx = 0
fig, ax = plt.subplots()

#ax.set_xlim(0,1)
#ax.set_ylim(0,250)

for pt in [0,1,4,5]:
    x = radii[idx, pt]
    y = cmass[idx, pt]

    # Compute running average:
    N = 10
    x = x[N::N]
    y = y[N::N]

    myG = G.to(units.cm ** 3 * units.g ** -1 * units.s ** -2).value
    y = np.sqrt((myG * y) / x)
    
    # Add zero:
    x = np.concatenate((np.array([0]), x))
    y = np.concatenate((np.array([0]), y))

    x = x * units.cm.to(units.kpc)
    y = y * units.cm.to(units.km)

    ax.plot(x,y)
plt.show()

In [ ]:
%time v_circ, r = dataset_compute.compute_rotation_curves(snap)
r = r * units.cm.to(units.kpc)
v_circ = v_circ * units.cm.to(units.km)

In [ ]:
%time v1kpc = snap.get_subhalos("V1kpc")
v1kpc = v1kpc * units.cm.to(units.km)

In [ ]:
print(np.sum(v1kpc == 0))
print(np.arange(v1kpc.size)[v1kpc==0])

In [ ]:
%time data = snap.get_subhalos("Max_Vcirc", group="Extended")
vmax = data[:,0] * units.cm.to(units.km)
rmax = data[:,1] * units.cm.to(units.kpc)

In [ ]:
vmax_shit = snap.get_subhalos("Vmax")
rmax_shit = snap.get_subhalos("VmaxRadius")
vmax_shit = vmax_shit * units.cm.to(units.km)
rmax_shit = rmax_shit * units.cm.to(units.kpc)

In [ ]:
idx = 0
fig, ax = plt.subplots()

ax.set_xlim(0,50)
#ax.set_ylim(0,250)

ax.plot(r[idx], v_circ[idx])

ax.axhline(vmax[idx],linestyle='dashed')
ax.axvline(rmax[idx],linestyle='dashed')
ax.axhline(v1kpc[idx],linestyle='dashed',c='black')
ax.axvline(1,linestyle='dashed',c='black')
ax.axhline(vmax_shit[idx],linestyle='dashed',c='red')
ax.axvline(rmax_shit[idx],linestyle='dashed',c='red')

plt.show()